# Linear Program Formulations
Based on Lab 9 of ENGRI 1101 Fall 2019

**Objectives:**
- Practice in formulating linear programming problems
- Introduce the idea of linear relaxation
- Demonstrate the relationship between an integer program and its linear relaxation

**Brief description:**
   In this lab we will see examples of modeling various problems as Linear Programs (LPs), coding and solving them with OR-Tools, and start considering the question of what can be done when the desired output of a mathematical model should be integer.

## Part 0: Preliminaries
This lab involves formulating two linear programs and solving them in OR-Tools. Formulating linear programs is tricky, and it is a skill best taught through blood, sweat, and tears. You are HIGHLY encouraged to work in small groups of 2-3 on this lab and to brainstorm and debate formulations together. If you’re unsure of how to formulate something, get help from your TA!

## Part 1: A Work-scheduling problem
[Adapted from Hillier & Lieberman] A new company is being set up to take telephone orders. The phone lines are staffed around the clock. People are hired to work in 8-hour shifts, starting at 6AM, 8AM, 12PM, 4PM, or 10PM. From survey data, the management has concluded that there are minimum staffing requirements at certain times of the day, as given in the table below.

| Time period   | Minimum Staff Size |
|---------------|--------------------|
| 6AM-8AM       | 48                 |
| 8AM-10AM      | 79                 |
| 10AM-Noon     | 65                 |
| Noon-2PM      | 87                 |
| 2PM-4PM       | 64                 |
| 4PM-6PM       | 87                 |
| 6PM-8PM       | 64                 |
| 8PM-10PM      | 73                 |
| 10PM-Midnight | 82                 |
| Midnight-6AM  | 43                 |

A person is paid differently, depending on their assigned shift, where the daily salary is 170, 160, 175, 180, and 195, for the five shifts, respectively. The management would like to decide the number of people to be hired for each shift so as to minimize the total salary costs.
The first part of this lab is aimed at helping you to formulate this problem as a linear programming problem. As usual, the first step is to make a first attempt at stating the decision variables. Conveniently, you did this in the prelab already!

**Q1:** Let’s call the variables $x_1, x_2, x_3, x_4,$ and $x_5$. Explain what each of these variables represents.

**A:** 

| Variable      | Meaning                                |
|---------------|----------------------------------------|
| $x_1$         |   |    
| $x_2$         |   | 
| $x_3$         |   | 
| $x_4$         |   | 
| $x_5$         |   | 

**Q2:** In terms of these decision variables, express the objective function.

**A:**

**Q3:** In your own words (i.e., using sentences and not mathematical notation) briefly describe what conditions must be satisfied for a given assignment of values of your decision variables to correspond to a feasible solution.

**A:**

**Q4:** Now try to express these restrictions as linear constraints in your decision variables. As a start, consider the period from 6AM to 8AM. You are required to staff this period with at least 48 people. Which shifts work during this time? How should you write this as a constraint?

**A:** 

**Q5:** Now complete your linear programming formulation (write the rest of the constraints).

**A:** 

**Q6:** If deleting a constraint from an LP does not create any new feasible solutions, then the constraint is called *redundant*. Why is your constraint for the 10PM-midnight shift redundant? Feel free to cross it out.

**A:**

Let’s look at the model for this problem in OR-Tools.

In [ ]:
# import OR-Tools
from ortools.linear_solver import pywraplp as OR

# define the model
mod = OR.Solver('work_sched', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# decision variables
x = {i: mod.NumVar(0, mod.infinity(), 'x_'+str(i)) for i in range(1,6)}
#x = {i: mod.IntVar(0, mod.infinity(), 'x_'+str(i)) for i in range(1,6)}

# objective function
mod.Minimize(170*x[1] + 160*x[2] + 175*x[3] + 180*x[4] + 195*x[5])

# constraints
mod.Add(x[1]                             >= 48)
mod.Add(x[1] + x[2]                      >= 79)
mod.Add(x[1] + x[2]                      >= 65)
mod.Add(x[1] + x[2] + x[3]               >= 87)
mod.Add(       x[2] + x[3]               >= 64)
mod.Add(              x[3] + x[4]        >= 87)
mod.Add(              x[3] + x[4]        >= 67) 
mod.Add(                     x[4]        >= 73)
mod.Add(                     x[4] + x[5] >= 82) 
mod.Add(                            x[5] >= 43);
#mod.Add(30*x[1] + 10*x[2] + 10*x[3] + 30*x[4] + 50*x[5] >= 6500); # mail order

Study the model and ensure that you understand how it corresponds to the model you came up with. Note that in Python, you can use # to comment out parts: everything that appears after the # on a line is ignored. Also note that the spacing in defining the constraints is not neccesary.

**Q7:** Did you write down the correct formulation (Ignore commented out lines)? If not, what did you do wrong?

**A:** 

Now you will solve this model using OR-Tools. Run the cell below.

In [ ]:
# solve and print solution
mod.Solve()
print('Objective =', mod.Objective().Value())
print('Solution:')
for v in mod.variables():
    print(v.name(),':', v.solution_value())

**Q8:** What is the objective function value of the optimal solution? What is the optimal solution (i.e., the values of the decision variables)?

**A:** 

Each staff member is also expected to take care of processing a number of mail-in orders when not on the telephone. Each person in the 6am, 8am, noon, 4pm and 10pm shifts are expected to handle 30, 10, 10, 30, and 50 orders, respectively. The staffing is required to be sufficient to handle 6500 orders. This constraint has been included in the model (the line labeled mail order) but it was commented out. Uncomment it now by removing the #, thereby adding the constraint to our model. Run the cell containing the model to update it. Get the new solution.

In [ ]:
# solve and print solution
mod.Solve()
print('Objective =', mod.Objective().Value())
print('Solution:')
for v in mod.variables():
    print(v.name(),':', v.solution_value())

**Q9:** Report the new solution below.

**A:** 

**Q10:** The solution to this model is not actually viable as a real-world hiring policy. Why not? Can you tweak the solution you found to give a viable (but not necessarily optimal) hiring plan? What is the total cost of your plan? 

**A:** 

In effect, the issue in the previous question was that we have an integer programming (IP) problem, not a linear programming (LP) problem. In the model file, comment out the line ```x = {i: mod.NumVar...``` and uncomment the line ```x = {i: mod.IntVar...```. This turns our LP into an IP. Run the cell containing the model to update it. Get the new solution.

In [ ]:
# solve and print solution
mod.Solve()
print('Objective =', mod.Objective().Value())
print('Solution:')
for v in mod.variables():
    print(v.name(),':', v.solution_value())

**Q11:** Write down the optimal solution and its value below. How does the optimal value of the integer program compare to the optimal value of the linear programming problem, its so-called linear relaxation? How does the optimal IP solution compare with the solution you proposed above?

**A:** 

## Part 2: A Production Process Model
[from Winston] Sunco Oil has three different processes that can be used to manufacture various types of gasoline. Each process involves blending oils in the company’s catalytic cracker. 
- Running process 1 for an hour costs 5 dollars and requires 2 barrels of crude oil 1 and 3 barrels of crude oil 2. The output of running process 1 for an hour is 2 barrels of gas 1 and 1 barrel of gas 2. 
- Running process 2 for an hour costs 4 dollars and requires 1 barrel of crude 1 and 3 barrels of crude 2. The output from running process 2 for an hour is 3 barrels of gas 2. 
- Running process 3 for an hour costs 1 dollar and requires 2 barrels of crude 2 and 3 barrels of gas 2. The output from running process 3 for an hour is 2 barrels of gas 3. 

Each week, 200 barrels of crude 1, at 2 dollars/barrel, and 300 barrels of crude 2, at 3 dollars/barrel, may be purchased. All gas produced can be sold at the following per-barrel prices: gas 1, 9; gas 2, 10; gas 3, 24. The catalytic cracker can be used for only 95 hours each week. Sunco Oil wishes to devise a weekly production plan that maximizes the production’s profit (revenues minus costs).

**Q12:** The above paragraph is pretty dense. Draw a pictorial representation of the flow of materials in this problem: from crude oil, to processes, to barrels of gas for sale. It might help to have one picture for each process (e.g. one picture for process 1 indicating what materials it needs to run and what materials it spits out). You might also want a chart indicating how much each barrel of crude oil / each process costs, as well as how much we profit from each gas.

**A:**

You’ll spend the rest of lab thinking about how to formulate this problem as a linear program. There are many ways to do this, but the most straightforward is to use 8 decision variables. Three of these are $\text{proc}_1$,$\text{proc}_2$ and $\text{proc}_3$, the numbers of hours spent running processes 1, 2 and 3. 

**Q13:** You should also have decision variables $\text{crude}_1$, $\text{crude}_2$, $\text{gas}_1$, $\text{gas}_2$, and $\text{gas}_3$. What do $\text{crude}_i$ and $\text{gas}_j$ represent?

**A:** 

**Q14:** State the objective function in terms of these variables: be sure to account for the cost of running processes and buying crude oil, as well as the profit from selling gas.

**A:**

In this problem, the primary decision variables are the $\text{proc}_i$ variables. We’ll need constraints that enforce that the other decision variables (the $\text{crude}_i$ and $\text{gas}_j$) line up with how much of each process we use.

**Q15:** For example, if we use two hours of process 1, five hours of process 2, and two hours of process 3, how much of each crude oil did we use? How much of each gas do we have to sell at the very end? (Hint: Be careful about the amount of gas 2 we have to sell at the end, since some of it will be used up in process 3.)

**A:** 

**Q16:** Based on the previous question, why does a constraint like the following make sense:
$$\text{crude}_1 = 2\text{proc}_1 + \text{proc}_2$$

**A:** 

**Q17:** Write four more equality constraints like the one above, one for each of $\text{crude}_2$, $\text{gas}_1$, $\text{gas}_2$, and $\text{gas}_3$ These constraints enforce that these extra decision variables represent what we want them two, based solely on how much time we spend on each process. (Hint: Your constraint for $\text{gas}_2$ will be a little tricky. You want $\text{gas}_2$ to represent the total amount of gas 2 that you have at the end of all processes, since this is what you sell for profit. However, some of gas 2 might be used up by process 3. Hence your gas 2 constraint will have three terms: two indicating how much you get from processes 1 and 2, and one indicating how much you lose from process 3.)

**A:** 

**Q18:** In addition to the five constraints written so far, we need 11 more: 8 of these are freebies (the non-negativity constraints on each of the eight decision variables). What other three constraints do we need? Think: what limits do we have from the problem statement?

**A:** 

**Q19:** Finish writing the model below and then solve it! As a sanity check to both your code and model, the optimal solution should leave you with a cost of 1,425 dollars and involve spending all time on process 2.

In [ ]:
# define the model
mod = OR.Solver('production_process', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# decision variables
proc = {i: mod.NumVar(0, mod.infinity(), 'proc_'+str(i)) for i in range(1,4)}
crude = {i: mod.NumVar(0, mod.infinity(), 'crude_'+str(i)) for i in range(1,3)}
gas = {i: mod.NumVar(0, mod.infinity(), 'gas_'+str(i)) for i in range(1,4)}

# objective function
mod.Maximize((9*gas[1] + 10*gas[2] + 24*gas[3])     # revenue from selling
             - (2*crude[1] + 3*crude[2])            # cost of crude oil
             - (5*proc[1] + 4*proc[2] + 1*proc[3])) # cost of running processes

# constraints
# TODO: Add the constraints. 
# Hint: the syntax is mod.Add(some expression)
# Hint: for equality constraints, use ==

# enforce the extra decison variables to represent what we want


# limit of oil that can bee purchased


# time limit



Now solve it!

In [ ]:
# solve and print solution
mod.Solve()
print('Objective =', mod.Objective().Value())
print('Solution:')
for v in mod.variables():
    print(v.name(),':', v.solution_value())